In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pylab

class RBM(nn.Module):
    '''
    class implementing the restricted boltzmann machine model
    '''

    def __init__(self, n_v, n_h):
        '''
        self.params[0] = W, weights
        self.params[1] = c, v bias
        self.params[2] = b, h bias
        '''
        super(RBM, self).__init__()
        self.params = [nn.Parameter(torch.randn(n_h, n_v)), nn.Parameter(torch.zeros(n_v)),
                       nn.Parameter(torch.zeros(n_h))]
        self.n_v = n_v
        self.n_h = n_h

    def forward(self, x, x_t, h_t):
        '''
        Forward function : p(x) = exp(-F(x)) F: Free energy
        '''
        F = - self.params[1].matmul(x) - nn.Softplus()(self.params[0].matmul(x).add_(self.params[2])).sum()
        E = - self.params[1].matmul(x_t) - self.params[2].matmul(h_t) - h_t.matmul(self.params[0].matmul(x_t))
        loss = F - E
        return loss

    def sample_h_knowing_x(self, x):
        '''

        :param x:
        :return: h
        '''
        probs = F.sigmoid(self.params[1] + self.params[0].matmul(x))
        h = B(probs).sample()
        return h

    def sample_x_knowing_h(self, h):
        '''
        :param h:
        :return: x
        '''
        probs = F.sigmoid(self.params[2] + self.params[0].matmul(h))
        x = B(probs).sample()
        return x

    def gibbs_sampling(self, num_itter, x_init, h_init):
        '''
        :param num_itter:
        :param x_init:
        :param h_init:
        :return: (x, h)
        '''

        for i in range(num_itter+1):
            if i == 0 :
                x_t, h_t = x_init, h_init
            else:
                h_t = self.sample_h_knowing_x(x_t)
                x_t = self.sample_x_knowing_h(h_t)

        return x_t, h_t

In [2]:
rbm = RBM(2,2)
h_t = torch.ones(2)
x_t = torch.ones(2)
x = torch.ones(2)
out1 = rbm.forward(x,x_t,h_t)
print(out1)

tensor(-2.2315)


In [4]:
out1.backward()
print(rbm.params[0].grad)

tensor([[ 0.7753,  0.7753],
        [ 1.6493,  1.6493]])


In [6]:
rbm = RBM(2,2)
h_t = torch.ones(2)
x_t = torch.ones(2)
x = torch.ones(2)
out2 = rbm.forward(x,x_t,h_t)
out2.backward()
print(rbm.params[0].grad)

tensor([[ 0.0595,  0.0595],
        [ 0.3877,  0.3877]])


In [ ]:
rbm = RBM(2,2)
h_t = torch.ones(2)
x_t = torch.ones(2)
x = torch.ones(2)
out1 = rbm.forward(x,x_t,h_t)